In [1]:
import requests
import pandas as pd

def get_total_ids_from_geohash(g_hash="wy7ec"):
    print(f"🚀 Geohash '{g_hash}' 구역에서 전체 ID 수집을 시작합니다...")
    url = f"https://apis.zigbang.com/house/property/v1/items/onerooms?geohash={g_hash}&depositMin=0&rentMin=0"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Origin": "https://www.zigbang.com",
        "Referer": "https://www.zigbang.com/"
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"❌ 접속 에러: {response.status_code}")
            return []
        data = response.json()
        raw_items = data.get("items", [])
        collected_data = []
        for item in raw_items:
            collected_data.append({
                "id": item.get("id"),
                "lat": item.get("lat"),
                "lng": item.get("lng")
            })
        print(f"✅ 수집 완료! 총 {len(collected_data)}개의 아이템 정보를 확보했습니다.")
        return collected_data
    except Exception as e:
        print(f"❌ 에러 발생: {e}")
        return []

# 실행 및 저장
raw_data = get_total_ids_from_geohash()
if raw_data:
    df_raw = pd.DataFrame(raw_data)
    df_raw.to_csv("zigbang_raw_ids.csv", index=False)

🚀 Geohash 'wy7ec' 구역에서 전체 ID 수집을 시작합니다...
✅ 수집 완료! 총 1107개의 아이템 정보를 확보했습니다.


In [4]:
def filter_yeongnam_core_items(input_file=r"C:\minwoin\zicbangcrawl\zigbang_raw_ids.csv", output_file="zigbang_2.csv"):
    print(f"🧹 '{input_file}' 파일에서 영남대역 핵심 매물 필터링을 시작합니다...")
    try:
        df = pd.read_csv(input_file)
        FIXED_BOUNDS = {
            'min_lat': 35.8355, 'max_lat': 35.8435,
            'min_lon': 128.7500, 'max_lon': 128.7610
        }
        mask = (
            (df['lat'] >= FIXED_BOUNDS['min_lat']) & (df['lat'] <= FIXED_BOUNDS['max_lat']) &
            (df['lng'] >= FIXED_BOUNDS['min_lon']) & (df['lng'] <= FIXED_BOUNDS['max_lon'])
        )
        filtered_df = df[mask].copy()
        filtered_df = filtered_df.drop_duplicates(subset=['id'])
        filtered_df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"✅ 필터링 완료! {len(df)}개 -> {len(filtered_df)}개 저장됨")
    except Exception as e:
        print(f"❌ 에러 발생: {e}")

# 실행
filter_yeongnam_core_items()

🧹 'C:\minwoin\zicbangcrawl\zigbang_raw_ids.csv' 파일에서 영남대역 핵심 매물 필터링을 시작합니다...
✅ 필터링 완료! 1107개 -> 607개 저장됨


In [ ]:
import time

def collect_perfect_details_v3():
    print("🚀 영남대역 매물 전수 조사를 시작합니다. (V3 정밀 모드)")
    try:
        df_base = pd.read_csv(r"C:\minwoin\zicbangcrawl\zigbang_2.csv")
        item_ids = df_base['id'].astype(str).tolist()
    except Exception as e:
        print(f"❌ 로드 실패: {e}"); return

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
    final_results = []
    
    for idx, item_id in enumerate(item_ids, 1):
        url = f"https://apis.zigbang.com/v3/items/{item_id}?version=&domain=zigbang"
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                data = response.json()
                item = data.get("item", {})
                price = item.get("price", {})
                loc = item.get("location", {})
                res = {
                    "매물번호": item_id, "보증금": price.get("deposit"), "월세": price.get("rent"),
                    "층": item.get("floor", {}).get("floor"), "위도": loc.get("lat"), "경도": loc.get("lng"),
                    "주소": item.get("addressOrigin", {}).get("fullText", "").strip()
                }
                pois = item.get("neighborhoods", {}).get("nearbyPois", [])
                for poi in pois:
                    if poi.get("exists"):
                        res[f"{poi.get('poiType')}_거리(m)"] = poi.get("distance")
                final_results.append(res)
                if idx % 50 == 0: print(f"✅ {idx}/{len(item_ids)} 완료...")
            time.sleep(0.5)
        except Exception as e:
            print(f"❌ 에러: {item_id} - {e}")

    df_final = pd.DataFrame(final_results)
    df_final.to_csv("zigbang.csv_dup", index=False, encoding="utf-8-sig")
    print("✨ 상세 수집 완료!")

# 실행
collect_perfect_details_v3()

🚀 영남대역 매물 전수 조사를 시작합니다. (V3 정밀 모드)


KeyboardInterrupt: 

In [ ]:
def finalize_data(input_file='zigbang_dup.csv', output_file='zigbang.csv'):
    try:
        df = pd.read_csv(input_file)
        # 도보 시간 계산 (80m 기준)
        if '지하철역_거리(m)' in df.columns:
            df['지하철역_도보(분)'] = (df['지하철역_거리(m)'] / 80).round().astype(int)
        
        # 중복 좌표 제거
        initial_count = len(df)
        df_unique = df.drop_duplicates(subset=['위도', '경도'], keep='first')
        
        df_unique.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"📊 최종 결과: {initial_count}건 -> {len(df_unique)}건 (중복 제거 완료)")
        print(f"💾 '{output_file}' 저장 완료.")
    except Exception as e:
        print(f"❌ 에러 발생: {e}")

# 실행
finalize_data()